In [1]:
!pip install transformers datasets faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 57.2 MB/s eta 0:00:00


Documentos

In [2]:
documents = [
    "O RAG combina recuperação de textos com modelos de linguagem.",
    "HuggingFace oferece ferramentas para modelos de NLP.",
    "FAISS é uma biblioteca usada para busca vetorial eficiente.",
    "Transformers são modelos avançados de deep learning para linguagem natural."
]


Models / FAISS

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
doc_embeddings = embedder.encode(documents, convert_to_numpy=True)

dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo de geração

In [4]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Função RAG

In [5]:
def rag_answer(question):
    q_emb = embedder.encode([question], convert_to_numpy=True)

    k = 2
    distances, indices = index.search(q_emb, k)

    retrieved_docs = "\n".join([documents[i] for i in indices[0]])

    prompt = f"""Use as informações abaixo para responder a pergunta.
Documentos:
{retrieved_docs}

Pergunta: {question}
Resposta:"""

    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(**inputs, max_new_tokens=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)


TESTES

In [6]:
print(rag_answer("Para que serve o FAISS?"))

Use as abaixo information to respond to the question. Documents: FAISS é uma biblioteca usada para busca vetorial eficiente. O RAG combina recuperaço de textos com modelos de linguagem.


In [7]:
print(rag_answer("O que é RAG?"))

Recomenda as informaçes abaixo para responder a question. Documentos: FAISS é uma biblioteca usada para busca vetorial eficiente. O RAG combina recuperaço de textos com modelos de linguagem.


In [8]:
print(rag_answer("O que são transformers na área de IA?"))

Modelos avançados de deep learning para linguagem natural
